# **Сети Колмогорова-Арнольда (KAN)**
---
## **1. Историческое развитие**

Теорему о представлении непрерывных функций нескольких переменных через суперпозиции непрерывных функций одной переменной и сложения, вставшую в основу KAN, сформулировал Андрей Колмогоров в 1957 году. В 1963 году Владимир Арнольд уточнил представление Колмогорова, показав, как именно можно построить функции от одной переменной, тем самым Арнольд придал теореме Колмогорова практическую форму. Именно этот результат стал известен как теорема Колмогорова-Арнольда.

Эта теорема, сформулированная в середине XX века, долгое время считалась теоретически значимой, но практически бесполезной из-за сложности реализации и негладкости внутренних функций, значительно усложняющих обучение.

---

## **2. Математические основы**

### **2.1 Теорема Колмогорова-Арнольда**

Из теоремы Колмогорова-Арнольда следует, что если $f$ — многомерная непрерывная функция в ограниченной области, то $f$ можно записать как конечную композицию непрерывных функций одной переменной и бинарной операции сложения следующим образом: $$f(x_1, \dots, x_n) = \sum_{q=1}^{2n+1} \Phi_q \left( \sum_{p=1}^{n} \phi_{q,p}(x_p) \right)
$$

### **2.2 Проблемы теоремы Колмогорова-Арнольда**

Теорему до недавнего времени не рассматривали в качестве основы для создания нейронных сетей по двум причинам:

1. **Негладкость одномерных функций.**
Теорема предполагает разложение многомерной функции на суперпозицию одномерных, но эти одномерные функции могут быть негладкими. Это делает невозможным применение метода обратного распространения ошибки (backpropagation), который требует дифференцируемости функций.

2. **Жёсткая фиксация структуры.**
В оригинальной теореме архитектура сети строго ограничена: всего два слоя и $2n + 1$ нейронов в скрытом слое (где $n$ — размерность входных данных). Такая структура слишком проста для большинства практических задач и не обладает гибкостью, необходимой для адаптации к сложным данным.

### **2.3 Решение проблем теоремы Колмогорова-Арнольда в KAN**

1. **B-сплайны.** Вместо произвольных функций KAN используют гладкие B-сплайны для представления активационных функций. Это гарантирует их дифференцируемость и позволяет применять современные методы обучения. Каждая активационная функция имеет вид:

$$
\phi(x) = w_b \cdot b(x) + w_s \cdot \mathrm{spline}(x)
$$,

$b(x)$ - базовая гладкая функция, $\mathrm{spline}(x)$ - линейная комбинация B-сплайнов:

$$
\mathrm{spline}(x) = \sum_{i} c_i B_i(x)
$$

2. **Глубокая композиция.** KAN вводят понятие слоя как матрицы функций:

$$
Φ_l = \{\phi_{l,j,i}\}, \quad i = 1..n_{in}, \quad j = 1.n_{out}
$$

где $\phi_{l,j,i}$ - обучаемая 1D функция параметризованная сплайном.

Выход KAN вычисляется как композиция L таких слоев:

$$
\text{KAN}(\mathbf{x}) = \left( \Phi_{L-1} \circ \Phi_{L-2} \circ \cdots \circ \Phi_0 \right) \mathbf{x}
$$

Что также можно нагляднее представить как:

$$
f(\mathbf{x}) = \sum_{i_{L-1}} \phi_{L-1, i_L, i_{L-1}} \left(
    \sum_{i_{L-2}} \cdots \left(
        \sum_{i_1} \phi_{1, i_2, i_1} \left(
            \sum_{i_0} \phi_{0, i_1, i_0}(x_{i_0})
        \right)
    \right) \cdots
\right)
$$

---

## **3. Основная структура**


### **3.1 Представление слоев**
Архитектура задаётся списком:

$$
[n_0, n_1, \dots, n_L]
$$

где $n_l$ - количество нейронов в слое $l$.

### **3.2 Функция активации**
Каждая связь между нейронами $(l, i)$ и $(l + 1, j)$ имеет свою функцию активации:

$$
\phi_{l,j,i}: ℝ → ℝ
$$

Параметризованную как:

$$
\phi_{l,j,i}(x) = w_b \cdot b(x) + w_s \cdot \mathrm{spline}(x)
$$

### **3.3 Вычисления в слое**

Выход нейрона (в данном случае нейрона $(l +1, j)$) вычисляется как сумма входящих в него функций:

$$
x_{l+1, j} = \sum_{i=1}^{n_l}\phi_{l,j,i}(x_{l, i})
$$

Если выражать это в матричной форме:

$$
x_{l+1} = Φ_lx_l
$$

где - функциональная матрица:

$$
\Phi_l =
\begin{bmatrix}
\phi_{l,1,1} & \cdots & \phi_{l,1,n_l} \\
\vdots & \ddots & \vdots \\
\phi_{l,n_{l+1},1} & \cdots & \phi_{l,n_{l+1},n_l}
\end{bmatrix}
$$

### **3.4 Полная сеть**

Полная сеть уже была описана выше и выглядит следующим образом:

$$
\text{KAN}(\mathbf{x}) = \left( \Phi_{L-1} \circ \Phi_{L-2} \circ \cdots \circ \Phi_0 \right) \mathbf{x}
$$

Или в другой форме:

$$
f(\mathbf{x}) = \sum_{i_{L-1}} \phi_{L-1, i_L, i_{L-1}} \left(
    \sum_{i_{L-2}} \cdots \left(
        \sum_{i_1} \phi_{1, i_2, i_1} \left(
            \sum_{i_0} \phi_{0, i_1, i_0}(x_{i_0})
        \right)
    \right) \cdots
\right)
$$

---

## 4. **Сравнение KAN с MLP**

Многослойные перцептроны долгое время остаются стандартом для аппроксимации нелинейных функций в глубоком обучении. Благодаря универсальной теореме аппроксимации и простоте реализации, MLP легли в основу практически всех современных нейросетевых архитектур.

### 4.1 **Функции активации**
- **MLP**: Нелинейные функции активации (ReLU, tanh, сигмоида и т.д.) одинаковые для всего слоя, расположены на узлах.
- **KAN**: 	Обучаемые нелинейные B-сплайны разные для каждого входа, расположены на ребрах.

### 4.2 **Параметризация**
- **MLP**:
  - Линейные веса: $W \in \mathbb{R}^{n_{l+1} \times n_l}$
  - Фиксированные активации: $\sigma(Wx + b)$
- **KAN**:
  - Обучаемые функции: $\phi_{l,j,i}(x) = w_b b(x) + w_s \text{spline}(x)$
  - Нет линейных весов - только параметры сплайнов

### 4.3 **Стратегия аппроксимации**

* **MLP**: Глобальная аппроксимация — каждый нейрон участвует в формировании всего выходного пространства, обучение распределяет информацию по всей сети.
* **KAN**: Локальная аппроксимация — B-сплайны действуют в ограниченных интервалах, что позволяет точно аппроксимировать функции с локальными особенностями.

### 4.4 **Метод обучения**

* **MLP**: Сквозное обучение (end-to-end) методом обратного распространения ошибки (backpropagation).
* **KAN**: Также используется backpropagation, благодаря дифференцируемости сплайнов.

### 4.5 **Скорость обучения**

* **MLP**: Обычно быстрее обучается благодаря простоте операций и хорошо проработанным оптимизаторам.
* **KAN**: Медленнее из-за более сложной архитектуры и численно затратных операций со сплайнами.

### 4.6 **Размер и вычислительная эффективность**

* **MLP**: Требует больше параметров, но оптимален по числу FLOPs (Floating Point Operations).
* **KAN**: Для достижения сопоставимой точности требует меньше параметров, но больше вычислений из-за стоимости сплайнов (больше FLOPs на одно соединение).

### 4.7 **Интерпретируемость**

* **MLP**: Низкая — веса и активации сложно интерпретировать, особенно в глубоких сетях.
* **KAN**: Выше — каждая функция на ребре явно задана (например, через B-сплайн), что позволяет анализировать вклад каждого входа в выход.

---

## **5. Методология обучения KAN**

Обучение KAN строится на тех же принципах, что и обучение классических нейросетей, но с рядом особенностей, связанных с обучаемыми функциями активации и нестандартной структурой параметров.

### **5.1 Инициализация параметров**

Обучение начинается с случайной инициализации параметров сети. В отличие от MLP, где инициализируются веса и смещения, в KAN инициализируются:

* коэффициенты B-сплайнов,
* весовые коэффициенты линейной составляющей (если используется),
* параметры shortcut-ветви (например, линейное преобразование после SiLU).

### **5.2 Прямой и обратный проход**

Процесс обучения включает два этапа:

1. **Прямой проход (forward pass)**: входные данные проходят через сеть, на каждом ребре применяются индивидуальные B-сплайны, результат агрегируется и формирует выход модели.
2. **Обратный проход (backward pass)**: вычисляется ошибка (loss) между предсказанием и истинной меткой, после чего с помощью правила цепочки (chain rule) дифференцирования вычисляются градиенты по всем параметрам.

Параметры обновляются с помощью стандартных методов оптимизации, таких как градиентный спуск, стохастический градиентный спуск (SGD) или Adam.

### **5.3 Проблемы стабильности и регуляризация**

Одной из ключевых сложностей при обучении KAN является обеспечение стабильности и сходимости процесса оптимизации. Это связано с тем, что обучаемые функции активации (B-сплайны) имеют сложную зависимость от параметров и могут вызывать переобучение или нестабильные градиенты.

Для решения этих проблем применяются:

* **Dropout** — для предотвращения переобучения;
* **Weight decay** — для ограничения роста параметров;
* **Batch normalization** и **Layer normalization** — для стабилизации распределений активаций и ускорения сходимости.

Кроме того, выбор функции потерь, инициализация диапазона B-сплайнов и настройка learning rate играют решающую роль в эффективности обучения.

### **5.4 Сквозное обучение**

KAN поддерживают сквозное (end-to-end) обучение, при котором все параметры сети обучаются одновременно. Это позволяет интегрировать KAN в современные фреймворки глубокого обучения (например, PyTorch), использовать его вместе с другими архитектурными блоками (CNN, Transformer) и обучать модель без ручной настройки отдельных этапов.

---

Если нужно, я могу добавить схему или график, поясняющий прямой и обратный проход в KAN.





## **Источники**

[1] Liu, Z. (2024). Kolmogorov–Arnold Networks (KANs). arXiv:2404.19756

[2] Runpeng Yu, Weihao Yu, and Xinchao Wang (2024). KAN or MLP: A Fairer Comparison. arXiv:2407.16674